# 逻辑回归练习（logistic regression practice）

## 项目背景

#### 项目背景：
* 1.项目基于吴达恩教授的《机器学习》课程。
* 2.数据均来源于课程配套资料。


#### 项目目的：
* 1、使用二分类逻辑回归预测学生能否录取。
* 2、使用正则化逻辑回归预测工厂的芯片能否通过质检。
* 3、熟悉逻辑回归的代价函数、梯度下降函数。
* 4、熟练运用正规化避免过度拟合的问题。


#### 二分类逻辑回归：
* 1、分析目的：通过学生两场考试的成绩和过往的录取情况预测学生能否录取。
* 2、数据说明：ex2data1
    * 第1列：第一场考试成绩
    * 第2列：第二场考试成绩
    * 第3列：学生的录取情况


#### 正则化逻辑回归：
* 1、分析目的：通过工厂芯片的两个检查结果和是否通过质检获得决策模型和边界。
* 2、数据说明：ex2data2
    * 第1列：第一个检查结果
    * 第2列：第二个检查结果
    * 第3列：是否通过质检

# 导入相关的库

In [356]:
import numpy as np
import pandas as pd
import tensorflow as tf
import plotly as py
import plotly.graph_objs as go
import cufflinks as cf
from plotly.offline import iplot,init_notebook_mode
cf.go_offline(connected=True)
init_notebook_mode(connected=True)
cf.set_config_file(theme='pearl') #设置一下制图颜色

# 1.二分类逻辑回归

## 1）读取数据并理解

In [357]:
#读取数据并赋予列名
df = pd.read_csv('D:\Learning\python\ML\code\ex2-logistic regression\ex2data1.txt',names=['first_score','second_score','application_result'])

In [358]:
df.head()

,first_score,second_score,application_result
0,34.623660,78.024693,0
1,30.286711,43.894998,0
2,35.847409,72.902198,0
3,60.182599,86.308552,1
4,79.032736,75.344376,1


In [359]:
df.info()
# 数据共100行，无null值，分数均为浮点型，是否录取为整数型

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   first_score         100 non-null    float64
 1   second_score        100 non-null    float64
 2   application_result  100 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 2.5 KB


In [360]:
df.describe()
# 数据无异常值

,first_score,second_score,application_result
count,100.000000,100.000000,100.000000
mean,65.644274,66.221998,0.600000
std,19.458222,18.582783,0.492366
min,30.058822,30.603263,0.000000
25%,50.919511,48.179205,0.000000
50%,67.032988,67.682381,1.000000
75%,80.212529,79.360605,1.000000
max,99.827858,98.869436,1.000000


In [361]:
np.sum(df.duplicated())
# 数据无重复值

0

## 2）数据可视化（data visualization）

In [362]:
data = [go.Scatter(
        x=df['first_score'],
        y=df['second_score'],
        mode='markers',
        marker=dict(
            sizemin=10,
            symbol=df['application_result'],#散点类型
            color=df['application_result'],#散点颜色
            line=dict(color='black', width=1.2)))]
figure = go.Figure(
    data=data,
    layout=go.Layout(
        height=500,
        width=500,
        xaxis=dict(title='first_score'),
        yaxis=dict(title='second_score')))
iplot(figure)

* 从图中可以看出，录取与否两个分类当中有着清晰的决策边界。

## 3）构建假设函数（Hypothesis）
g 代表一个常用的逻辑函数（logistic function），为S形函数（Sigmoid function），公式为： \\[g\left( z \right)=\frac{1}{1+{{e}^{-z}}}\\] 
合起来，逻辑回归模型的假设函数： 
	\\[{{h}_{\theta }}\left( x \right)=\frac{1}{1+{{e}^{-{{\theta }^{T}}X}}}\\] 

In [363]:
# 定义sigmoid function
def sigmoid(z):
    g=1 / (1 + np.exp(-z))
    return g

In [364]:
# 我们来看一下S函数长啥样
n = np.arange(-10, 10, step=1)
s = sigmoid(n)

data=[go.Scatter(x=n.tolist(),y=s.tolist(),mode='lines')]
iplot(data)

## 4）构建代价函数（Cost Function）
> * $max(\ell(\theta)) = min(-\ell(\theta))$  
> * choose $-\ell(\theta)$ as the cost function

$$\begin{align}
  & J\left( \theta  \right)=-\frac{1}{m}\sum\limits_{i=1}^{m}{[{{y}^{(i)}}\log \left( {{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)+\left( 1-{{y}^{(i)}} \right)\log \left( 1-{{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)]} \\ 
 & =\frac{1}{m}\sum\limits_{i=1}^{m}{[-{{y}^{(i)}}\log \left( {{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)-\left( 1-{{y}^{(i)}} \right)\log \left( 1-{{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)]} \\ 
\end{align}$$

In [365]:
# 定义theta、X、theta
df.insert(0, 'Ones', 1)

cols = df.shape[1]
X = df.iloc[:,0:cols-1]
y = df.iloc[:,cols-1:cols]

# convert to numpy arrays and initalize the parameter array theta
X = np.array(X.values)
y = np.array(y.values)
theta = np.zeros(3)
print(X.shape,y.shape,theta.shape)

(100, 3) (100, 1) (3,)


In [368]:
np.matrix(theta)

matrix([[0., 0., 0.]])

In [369]:
def cost(theta, X, y):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    return np.sum(first - second) / (len(X))

In [370]:
# 算出theta为（0,0,0）时的代价函数
cost(theta,X,y)

0.6931471805599453

## 5）梯度下降算法（Gradient Descent）
* 本项目使用批量梯度下降（batch gradient descent）  
* 转化为向量化计算： $\frac{1}{m} X^T( Sigmoid(X\theta) - y )$
$$\frac{\partial J\left( \theta  \right)}{\partial {{\theta }_{j}}}=\frac{1}{m}\sum\limits_{i=1}^{m}{({{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}})x_{_{j}}^{(i)}}$$

In [371]:
def gradient(theta, X, y):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)
    
    error = sigmoid(X * theta.T) - y
    
    for i in range(parameters):
        term = np.multiply(error, X[:,i])
        grad[i] = np.sum(term) / len(X)
    
    return grad

* 这是梯度下降的偏导数计算函数，实际上我们得到的仅是一个步长。

In [372]:
# 算出theta为（0,0,0）时的偏导数（步长）
grad=gradient(theta, X, y)
grad

array([ -0.1       , -12.00921659, -11.26284221])

* 实际上我们可以使用SciPy的“optimize”寻求最优参数，而不需要设置alpha。

In [373]:
import scipy.optimize as opt

In [374]:
result = opt.fmin_tnc(func=cost, x0=theta, fprime=gradient, args=(X, y))
result

(array([-25.16131872,   0.20623159,   0.20147149]), 36, 0)

In [375]:
# 我们来看一下最小代价函数是多少
cost(result[0], X, y)

0.20349770158947425

## 6）预测函数
\\[{{h}_{\theta }}\left( x \right)=\frac{1}{1+{{e}^{-{{\theta }^{T}}X}}}\\] 
当${{h}_{\theta }}$大于等于0.5时，预测 y=1

当${{h}_{\theta }}$小于0.5时，预测 y=0 。

In [376]:
def predict(theta, X):
    probability = sigmoid(X * theta.T)
    return [1 if x >= 0.5 else 0 for x in probability]

In [377]:
# 通过训练样本检测预测的准确性
final_theta = np.matrix(result[0])
predictions = predict(final_theta, X)
correct = [1 if ((a == 1 and b == 1) or (a == 0 and b == 0)) else 0 for (a, b) in zip(predictions, y)]
accuracy = (sum(map(int, correct)) % len(correct))
print ('accuracy = {0}%'.format(accuracy))

accuracy = 89%


* 说明运用这个假设预测学生能否被录取，准确了达到89%。
* 但实际上这是训练样本的准确性，实际预测的准确率可能要低一些。

## 7）决策边界
http://stats.stackexchange.com/questions/93569/why-is-logistic-regression-a-linear-classifier
> $X \times {{\theta }^{T}}= 0$  (this is the line)

In [474]:
coef=-(final_theta / final_theta[:,2])
x = np.arange(130, step=0.1)
y = coef[:,0] + coef[:,1]*x

In [475]:
scatter = go.Scatter(
        x=df['first_score'],
        y=df['second_score'],
        mode='markers',
        name='training data',
        marker=dict(
            sizemin=10,
            symbol=df['application_result'],#散点类型
            color=df['application_result'],#散点颜色
            line=dict(color='black', width=1.2)))
line=go.Scatter(x=x.tolist(),y=y.tolist()[0],mode='lines',name='descision boundary')
data=[scatter,line]
layout=go.Layout(height=500,width=500)
figure=go.Figure(data,layout)
py.offline.iplot(figure)

# 2.正则化逻辑回归
正则化是成本函数中的一个术语，它使算法更倾向于“更简单”的模型（在这种情况下，模型将更小的系数）,避免过度拟合。

## 1）读取数据并理解

In [447]:
df2 = pd.read_csv('D:\Learning\python\ML\code\ex2-logistic regression\ex2data2.txt', names=['test1', 'test2', 'accepted'])
df2.head()

,test1,test2,accepted
0,0.051267,0.69956,1
1,-0.092742,0.68494,1
2,-0.213710,0.69225,1
3,-0.375000,0.50219,1
4,-0.513250,0.46564,1


In [448]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   test1     118 non-null    float64
 1   test2     118 non-null    float64
 2   accepted  118 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 2.9 KB


In [449]:
df2.describe()

,test1,test2,accepted
count,118.000000,118.000000,118.000000
mean,0.054779,0.183102,0.491525
std,0.496654,0.519743,0.502060
min,-0.830070,-0.769740,0.000000
25%,-0.372120,-0.254385,0.000000
50%,-0.006336,0.213455,0.000000
75%,0.478970,0.646562,1.000000
max,1.070900,1.108900,1.000000


## 2）数据可视化

In [450]:
data = [go.Scatter(
        x=df2['test1'],
        y=df2['test2'],
        mode='markers',
        marker=dict(
            sizemin=10,
            symbol=df2['accepted'],#散点类型
            color=df2['accepted'],#散点颜色
            line=dict(color='black', width=1.2)))]
figure = go.Figure(
    data=data,
    layout=go.Layout(
        height=500,
        width=500,
        xaxis=dict(title='test1'),
        yaxis=dict(title='test2')))
iplot(figure)

* 从图中可以看出，这个数据没有线性决策边界，本次逻辑回归我们采用多项式进行拟合。

In [451]:
degree = 5
x1 = df2['test1']
x2 = df2['test2']

df2.insert(3, 'Ones', 1)

for i in range(1, degree):
    for j in range(0, i):
        df2['F' + str(i) + str(j)] = np.power(x1, i-j) * np.power(x2, j)

df2.drop('test1', axis=1, inplace=True)
df2.drop('test2', axis=1, inplace=True)

df2.head()

,accepted,Ones,F10,F20,F21,F30,F31,F32,F40,F41,F42,F43
0,1,1,0.051267,0.002628,0.035864,0.000135,0.001839,0.025089,0.000007,0.000094,0.001286,0.017551
1,1,1,-0.092742,0.008601,-0.063523,-0.000798,0.005891,-0.043509,0.000074,-0.000546,0.004035,-0.029801
2,1,1,-0.213710,0.045672,-0.147941,-0.009761,0.031616,-0.102412,0.002086,-0.006757,0.021886,-0.070895
3,1,1,-0.375000,0.140625,-0.188321,-0.052734,0.070620,-0.094573,0.019775,-0.026483,0.035465,-0.047494
4,1,1,-0.513250,0.263426,-0.238990,-0.135203,0.122661,-0.111283,0.069393,-0.062956,0.057116,-0.051818


## 3）正则化代价函数（regularized cost）
$$J\left( \theta  \right)=\frac{1}{m}\sum\limits_{i=1}^{m}{[-{{y}^{(i)}}\log \left( {{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)-\left( 1-{{y}^{(i)}} \right)\log \left( 1-{{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)]}+\frac{\lambda }{2m}\sum\limits_{j=1}^{n}{\theta _{j}^{2}}$$

In [456]:
# 定义正则化代价函数
def costreg(theta, X, y, reg_para):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (reg_para / (2 * len(X))) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / len(X) + reg

## 4）正则化梯度下降（regularized gradient descent）
\begin{align}
  & Repeat\text{ }until\text{ }convergence\text{ }\!\!\{\!\!\text{ } \\ 
 & \text{     }{{\theta }_{0}}:={{\theta }_{0}}-a\frac{1}{m}\sum\limits_{i=1}^{m}{[{{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}}]x_{_{0}}^{(i)}} \\ 
 & \text{     }{{\theta }_{j}}:={{\theta }_{j}}-a\frac{1}{m}\sum\limits_{i=1}^{m}{[{{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}}]x_{j}^{(i)}}+\frac{\lambda }{m}{{\theta }_{j}} \\ 
 & \text{          }\!\!\}\!\!\text{ } \\ 
 & Repeat \\ 
\end{align}

对上面的算法中 j=1,2,...,n 时的更新式子进行调整可得： 
${{\theta }_{j}}:={{\theta }_{j}}(1-a\frac{\lambda }{m})-a\frac{1}{m}\sum\limits_{i=1}^{m}{({{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}})x_{j}^{(i)}}$


In [457]:
# 定义正则化梯度下降函数
def gradientreg(theta, X, y, reg_para):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    
    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)
    
    error = sigmoid(X * theta.T) - y
    
    for i in range(parameters):
        term = np.multiply(error, X[:,i])
        
        if (i == 0):
            grad[i] = np.sum(term) / len(X)
        else:
            grad[i] = (np.sum(term) / len(X)) + ((reg_para / len(X)) * theta[:,i])
    
    return grad

In [458]:
# 定义X，y，theta，reg_para
cols = df2.shape[1]
X2 = df2.iloc[:,1:cols]
y2 = df2.iloc[:,0:1]

X2 = np.array(X2.values)
y2 = np.array(y2.values)
theta2 = np.zeros(11)
reg_para=1

In [459]:
# 计算theta为0时的代价
costreg(theta2,X2,y2,reg_para)

0.6931471805599454

In [461]:
# 计算theta为0时的步长
gradientReg(theta2, X2, y2, reg_para)

array([0.00847458, 0.01878809, 0.05034464, 0.01150133, 0.01835599,
       0.00732393, 0.00819244, 0.03934862, 0.00223924, 0.01286005,
       0.00309594])

In [467]:
# 使用优化函数计算最佳的theta
result2 = opt.fmin_tnc(func=costreg, x0=theta2, fprime=gradientreg, args=(X2, y2, reg_para))
final_theta2=np.matrix(result2[0])
final_theta2

matrix([[ 0.53010248,  0.29075567, -1.60725764, -0.58213819,  0.01781027,
         -0.21329508, -0.40024142, -1.37144139,  0.02264304, -0.9503358 ,
          0.0344085 ]])

## 5）预测函数

In [468]:
# 预测准确率
predictions = predict(final_theta2, X2)
correct = [1 if ((a == 1 and b == 1) or (a == 0 and b == 0)) else 0 for (a, b) in zip(predictions, y2)]
accuracy = (sum(map(int, correct)) % len(correct))
print ('accuracy = {0}%'.format(accuracy))

accuracy = 78%


## 鸣谢：
感谢黄海广博士提供的读书笔记及各项资料，我会在机器学习路上继续加油！